# An Adventure in Algorithms: Dipping Your Toes into Machine Learning

Many of the following demonstrations are from the [scikit-learn tutorials](https://scikit-learn.org/stable/index.html):  

## A Simple Linear Regression

In [ ]:
import matplotlib.pyplot as plt
import random
import math
import numpy as np

from sklearn import datasets, linear_model, neural_network, metrics, svm, neighbors, tree
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, LearningCurveDisplay

In [ ]:
seed = 9485

random.seed(seed)

m_orig = random.uniform(0.5, 3)
b_orig = random.uniform(-2,5)
N = 8
xmin = 0.5
xmax = 3

x = np.array([round(random.uniform(xmin, xmax), 2)for _ in range(N)])
y = np.array([round(m_orig*xi + b_orig + random.normalvariate(mu=0.0, sigma=.35),2) for xi in x])

plt.scatter(x, y, color='blue')
plt.xlim(0, 3)
plt.ylim(-2, 14)
plt.show()

In [ ]:
def plot_new_line(ms, bs, x, y):
    xs = np.array([0,3])

    # 2 Plots, side-by-side
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,4), sharey=True)

    # Add the most recent line
    ax1.plot(xs, ms[-1]*xs+bs[-1], color='red')

    # Add lines for the previous attempts
    for (i,(m, b)) in enumerate(zip(ms, bs)):
        color = 'red' if i == len(ms)-1 else 'gray'
        ax2.plot(xs, m*xs+b, color=color)

    # Draw the scatter plot on top of the lines, prevents the previous attempts from cluttering up the data
    ax1.scatter(x, y, color='blue')
    ax2.scatter(x, y, color='blue')
    plt.xlim(0, 3)
    plt.ylim(-2, 14)
    plt.show()

    return [mean_squared_error(y, m*x+b) for (m,b) in zip(ms, bs)]

ms=[]
bs=[]

Remember that a line is defined as $y = mx + b$, so we need to specify values for $m$ and $b$. These are called parameters of the model.

In [ ]:
m = 0.0
b = 0.0

ms.append(m)
bs.append(b)
mses = plot_new_line(ms, bs, x, y)
print('m         b         mse')
for m, b, mse in zip(ms, bs, mses):
    print('{m:.4f}    {b:.4f}    {mse:.5f}'.format(m=m, b=b, mse=mse))

## Let's Do This Automically

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(x.reshape(-1,1), y.reshape(-1,1))

# Plot the data
plt.scatter(x,y, color='blue')

# Plot the line which minimizes MSE
x_toplot = np.array([0,3]).reshape(-1,1)
y_toplot = regr.predict(x_toplot)
plt.plot(x_toplot, y_toplot, color='red')
plt.xlim(0, 3)
plt.ylim(-2, 14);

# Print the MSE and the parameters for the line
#print(f'MSE: {mean_squared_error(y, regr.predict(x.reshape(-1,1)))}')
#print(f'm = {regr.coef_[0][0]} \nb = {regr.intercept_[0]}')

## Back to Presentation for a Discussion About Training and Test Sets

## A More Practical Example

The following code loads a dataset that includes a "toy" dataset which is taken from a diabetes dataset. We are narrowing to one "feature" here (normalized BMI) and predicting a "quantitative measure of disease progression one year after baseline." The data comes from [this site](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html).

In the following plot, the black dots are the training set, while the red dots are the test set.

In [ ]:
# Load the diabetes dataset
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# Use only one feature (BMI, normalized)
diabetes_X = diabetes_X[:, np.newaxis, 2]

# Print the number of data points
print(f'Number of data points: {diabetes_X.shape[0]}')

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-88]
diabetes_X_test = diabetes_X[-88:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-88]
diabetes_y_test = diabetes_y[-88:]

# Print information about the training and testing sets
print(f'Number of data points in the training set: {diabetes_X_train.shape[0]}')
print(f'Number of data points in the test set: {diabetes_X_test.shape[0]}')

In [ ]:
# Plot the data (training set in black, test set in red)
plt.scatter(diabetes_X_train, diabetes_y_train, color="black")
plt.scatter(diabetes_X_test, diabetes_y_test, color="red")
plt.show()

Now, let's set up a linear regression

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_train, diabetes_y_train)

# Make predictions using the testing set
diabetes_y_pred = regr.predict(diabetes_X_test)

# The coefficients
print(f'MSE: {mean_squared_error(diabetes_y_test, diabetes_y_pred)}')
print(f'm = {regr.coef_[0]} \nb = {regr.intercept_}')

# For plotting the line of best fit, get the min and max x value so that we can build the line to match all the data
xmin = min(diabetes_X_train.min(), diabetes_X_test.min())
xmax = max(diabetes_X_train.max(), diabetes_X_test.max())
x_plot = np.array([xmin, xmax]).reshape(-1,1)
y_plot = regr.predict(x_plot)

# Plot outputs
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10, 4), sharey=True)
ax1.scatter(diabetes_X_train, diabetes_y_train, color="black")
ax2.scatter(diabetes_X_test, diabetes_y_test, color="red")
ax1.plot(x_plot, y_plot, color="blue", linewidth=3)
ax2.plot(x_plot, y_plot, color="blue", linewidth=3)

plt.show()

## Let's Talk About Neural Networks

1. Watch the 3blue1brown video
2. Play with the [tensorflow playground](http://playground.tensorflow.org)
3. Continue below where we create our own neural network to recognize digits

## A Neural Network for Recognizing Handwritten Digits

In [ ]:
# Load the MNIST dataset
digits = datasets.load_digits()

# Plot a few of the digits
_, axes = plt.subplots(nrows=2, ncols=8, figsize=(15, 5))
for ax, image, label in zip(axes.flatten(), digits.images, digits.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("Training: %i" % label)

In [ ]:
# flatten the images
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))/16

# Let's see the images as lists of numbers
print(f'data is {data.shape[0]} by {data.shape[1]}, in other words, we have {data.shape[0]} images, flattened down so that each has {data.shape[1]} numbers associated with it')
print(data)

#### Notice that the above are only 8x8 images with a total of 64 pixels, we are using the smaller size here so that the computation is faster and we're not waiting around for the computer to do the training

In [ ]:
# Create a training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.2, shuffle=True
)

# Print the number of training and testing samples
print(f'There are {X_train.shape[0]} samples in the training set.')
print(f'There are {X_test.shape[0]} samples in the test set.')

In [ ]:
# Set up some hyperparameters first - these control the structure of the neural network and how we train it
hidden_layer_sizes = (50)
activation = 'logistic' # Other options are 'relu', 'tanh', 'identity'
learning_rate = 0.01
max_iter = 700

# Create the Neural Network Classifier Object
mlp = neural_network.MLPClassifier(hidden_layer_sizes = hidden_layer_sizes, 
                                   activation = activation, 
                                   learning_rate_init = learning_rate, 
                                   max_iter = max_iter)

# Train the network
mlp.fit(X_train, y_train)

# Now predict the labels for the test set
predicted = mlp.predict(X_test)

# Let's visualize some predictions
_, axes = plt.subplots(nrows=2, ncols=8, figsize=(15, 5))
for ax, image, prediction in zip(axes.flatten(), X_test, predicted):
    ax.set_axis_off()
    image = image.reshape(8, 8)
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title(f"Prediction: {prediction}")


## Some Analysis of the Results

First, we'll start with some statistics:

In [ ]:
n_corr_preds_test = sum(y_test == predicted)
n_corr_preds_train = sum(y_train == mlp.predict(X_train))
print('Training set: ')
print(f'  {n_corr_preds_train} correct predictions')
print(f'  {y_train.shape[0] - n_corr_preds_train} incorrect predictions')
print(f'  Accuracy: {n_corr_preds_train*100/y_train.shape[0]}% correct')
print('Test set: ')
print(f'  {n_corr_preds_test} correct predictions')
print(f'  {y_test.shape[0] - n_corr_preds_test} incorrect predictions')
print(f'  Accuracy: {n_corr_preds_test*100/y_test.shape[0]}% correct')

#### We can also look at the confusion matrix

In [ ]:
disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, predicted)
disp.figure_.suptitle("Confusion Matrix for Test Set")
# print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

### Let's look at the samples that got mis-labeled:

In [ ]:
# Get the mislabeled images
mislabeled = (y_test != predicted)

# Plot them
n_rows = math.ceil(sum(mislabeled) / 8)
_, axes = plt.subplots(nrows=n_rows, ncols=8, figsize=(15, 5))
for ax, image, prediction, true_label in zip(axes.flatten(), X_test[mislabeled], predicted[mislabeled], y_test[mislabeled]):
    ax.set_axis_off()
    image = image.reshape(8, 8)
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title(f"Pred: {prediction}; True: {true_label}")

### Take another look at the training loss

## Can Other Models Do Any Better?

### First, The Support Vector Machine (SVM)

In [ ]:
gamma = 0.001

clf = svm.SVC(gamma=gamma)
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)

n_corr_preds_test = sum(y_test == predicted)
n_corr_preds_train = sum(y_train == mlp.predict(X_train))
print('Training set: ')
print(f'  {n_corr_preds_train} correct predictions')
print(f'  {y_train.shape[0] - n_corr_preds_train} incorrect predictions')
print(f'  Accuracy: {n_corr_preds_train*100/y_train.shape[0]}% correct')
print('Test set: ')
print(f'  {n_corr_preds_test} correct predictions')
print(f'  {y_test.shape[0] - n_corr_preds_test} incorrect predictions')
print(f'  Accuracy: {n_corr_preds_test*100/y_test.shape[0]}% correct')

disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, predicted)
disp.figure_.suptitle("Confusion Matrix for Test Set")
# print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

### How About K-Nearest Neighbors?

In [ ]:
n_neighbors = 3

clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors)
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)

n_corr_preds_test = sum(y_test == predicted)
n_corr_preds_train = sum(y_train == mlp.predict(X_train))
print('Training set: ')
print(f'  {n_corr_preds_train} correct predictions')
print(f'  {y_train.shape[0] - n_corr_preds_train} incorrect predictions')
print(f'  Accuracy: {n_corr_preds_train*100/y_train.shape[0]}% correct')
print('Test set: ')
print(f'  {n_corr_preds_test} correct predictions')
print(f'  {y_test.shape[0] - n_corr_preds_test} incorrect predictions')
print(f'  Accuracy: {n_corr_preds_test*100/y_test.shape[0]}% correct')

disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, predicted)
disp.figure_.suptitle("Confusion Matrix for Test Set")
# print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

### What About Decision Trees?

In [ ]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)

n_corr_preds_test = sum(y_test == predicted)
n_corr_preds_train = sum(y_train == mlp.predict(X_train))
print('Training set: ')
print(f'  {n_corr_preds_train} correct predictions')
print(f'  {y_train.shape[0] - n_corr_preds_train} incorrect predictions')
print(f'  Accuracy: {n_corr_preds_train*100/y_train.shape[0]}% correct')
print('Test set: ')
print(f'  {n_corr_preds_test} correct predictions')
print(f'  {y_test.shape[0] - n_corr_preds_test} incorrect predictions')
print(f'  Accuracy: {n_corr_preds_test*100/y_test.shape[0]}% correct')

disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, predicted)
disp.figure_.suptitle("Confusion Matrix for Test Set")
# print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

# Unsupervised Learning

First a simple example:

In [ ]:
from sklearn.datasets import make_blobs
X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

## How about a more practical example?

When images are stored on a computer, the standard "lossless" format requires being able to store 96,615 possible colors for each pixel. This creates a large file size, but maintains as much detail as possible.

It's often convenient to try and compress images. One of the popular formats is the Graphics Interchange Format (GIF). In this format, a pallete of 64 colors is chosen and all the pixels in the image are re-mapped to one of the 64 possible colors. Choosing those colors can be a complicated process, but one way to do it is to use a clustering algorithm (so that similar colors are in the same cluster). We implement this below.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import load_sample_image
from sklearn.metrics import pairwise_distances_argmin
from sklearn.utils import shuffle

n_colors = 64

# Load a sample image
pic = load_sample_image("china.jpg")

# Convert to floats instead of the default 8 bits integer coding. Dividing by
# 255 is important so that plt.imshow works well on float data (need to
# be in the range [0-1])
pic = np.array(pic, dtype=np.float64) / 255

# Load Image and transform to a 2D numpy array.
w, h, d = original_shape = tuple(pic.shape)
assert d == 3
image_array = np.reshape(pic, (w * h, d))

image_array_sample = shuffle(image_array, random_state=0, n_samples=1_500)
kmeans = KMeans(n_clusters=n_colors, random_state=0).fit(image_array_sample)

# Get labels for all points
labels = kmeans.predict(image_array)

def recreate_image(codebook, labels, w, h):
    """Recreate the (compressed) image from the code book & labels"""
    return codebook[labels].reshape(w, h, -1)

# Display all results, alongside original image
plt.figure(1)
plt.clf()
plt.axis("off")
plt.title("Original image (96,615 colors)")
plt.imshow(pic)

plt.figure(2)
plt.clf()
plt.axis("off")
plt.title(f"Quantized image ({n_colors} colors, K-Means)")
plt.imshow(recreate_image(kmeans.cluster_centers_, labels, w, h));

#### We can even show the color palette:

In [ ]:
plt.figure(figsize=(13,7))
plt.axis("off")
plt.imshow(kmeans.cluster_centers_.reshape(1, n_colors, -1));

# Reinforcement Learning

Check out [this video](https://youtu.be/kopoLzvh5jY?feature=shared).